# **IMPORT LIBRERIE**

In [56]:
import carla, time, random

from Adas import Forward_collision_warning_mqtt
from Adas_old import Forward_collision_warning_mqtt as Forward_collision_warning_mqtt_old

from enum import Enum

# **CARLA**

In [2]:
# Connessione
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

# Instanza mondo
world = client.get_world()

# Caricamento mappa
if not world.get_map().name == 'Carla/Maps/Town03':
    world = client.load_world('Town03')

KeyboardInterrupt: 

# **METODI PER LA CREAZIONE DI ATTORI**

In [3]:
def spawn_vehicle_by_spawnPoint(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [4]:
def spawn_vehicle_by_location(vehicle_index=0, location=carla.Location(), rotation = carla.Rotation()):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[vehicle_index]
    spawn_point = carla.Transform()
    spawn_point.location = location
    spawn_point.rotation = rotation
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [5]:
def spawn_walker():
    walker_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))

    start_location = carla.Location(x=70, y=25, z=1)

    trans = carla.Transform()
    trans.location = start_location

    return world.spawn_actor(walker_bp, trans)

# **FUNZIONI DI SUPPORTO**

In [ ]:
# Comandi guida veicolo
def move_forward(vehicle):
    global throttle
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)

def stop_vehicle(vehicle):
    print("stop")
#     control = carla.VehicleControl()
#     control.throttle = 0.0
#     control.brake = 1
#     control.hand_brake = True
#     vehicle.apply_control(control)

In [7]:
# Simulazione sensore umidità dell'asflato
def get_asphalt_friction_coefficient():
    world = client.get_world()
    weather = world.get_weather()
    weather.precipitation_deposits
    weather.wetness
    return 0.8

In [8]:
# Uscita dalla simulazione
def stop_simulation():
    global run
    run = False
    print("stop")
    
    

In [9]:
def get_vehicle_velocity(vehicle):
    velocity_vector = vehicle.get_velocity()
    velocity = (velocity_vector.x**2 + velocity_vector.y**2 + velocity_vector.z**2)**0.5
    if vehicle.get_control().reverse:
        velocity = -velocity
    return velocity

In [16]:
def destroy_all_vehicles(world):
    all_actors = world.get_actors()
    
    vehicles = all_actors.filter('vehicle.*')
    
    print(f"Numero di veicoli trovati: {len(vehicles)}")
    
    for vehicle in vehicles:
        vehicle.destroy()
        print(f"Veicolo distrutto: {vehicle.id}")

In [31]:
destroy_all_vehicles(world)

Numero di veicoli trovati: 1
Veicolo distrutto: 695


In [29]:
vehicle = spawn_vehicle_by_location(0, carla.Location(x=100, y=-6.9, z=1), carla.Rotation(yaw = 180))

# **SIMULAZIONE**

In [62]:
destroy_all_vehicles(world)

spawn_vehicle_by_location(0, carla.Location(x=100, y=-6.9, z=1), carla.Rotation(yaw = 180))

try:
    vehicle = spawn_vehicle_by_location(2, carla.Location(x=140, y=-5.8, z=1), carla.Rotation(yaw = 180))
#     vehicle = spawn_vehicle_by_spawnPoint()
    #walker = spawn_walker()

    run = True
    throttle = 0.7

    adas = Forward_collision_warning_mqtt(
        world = world,
        attached_vehicle = vehicle,
        get_asphalt_friction_coefficient = lambda : get_asphalt_friction_coefficient(),
        action_listener = lambda : stop_simulation(),
        min_ttc=0.5
    )

#     vehicle.set_autopilot(True)

    while run:
        move_forward(vehicle)
        time.sleep(0.05)
    
    stop_vehicle(vehicle)
    time.sleep(10)

finally:
    adas.destroy()
    vehicle.destroy()
    #walker.destroy()
    

Numero di veicoli trovati: 1
Veicolo distrutto: 762
Send `ACTION` to topic `carla/fcw_state`
stop
stop


In [36]:
import sys
sys.path.insert(0,'C:/svs/CARLA_0.9.15/WindowsNoEditor/PythonAPI/carla')
from agents.navigation.basic_agent import BasicAgent

In [32]:
def actor_vehicle(world, vehicle, desitination, speed = 60):
    world.wait_for_tick()
    actor_agent = BasicAgent(vehicle)
    actor_agent.set_destination(desitination)
    actor_agent.set_target_speed(speed)
    actor_agent.ignore_vehicles()
    world.wait_for_tick()
    return actor_agent

In [54]:
destroy_all_vehicles(world)

spawn_vehicle_by_location(0, carla.Location(x=100, y=-6.9, z=1), carla.Rotation(yaw = 180))

destination = carla.Location(x=100, y=-6.9, z=1)
try:
    vehicle = spawn_vehicle_by_location(2, carla.Location(x=140, y=-5.8, z=1), carla.Rotation(yaw = 180))
    world.wait_for_tick()

    actor_agent = actor_vehicle(world, vehicle, destination, 120)
    
    
    run = True
    throttle = 0.7

    adas = Forward_collision_warning_mqtt(
        world = world,
        attached_vehicle = vehicle,
        get_asphalt_friction_coefficient = lambda : get_asphalt_friction_coefficient(),
        action_listener = lambda : stop_simulation(),
        min_ttc=0.5
    )

#     vehicle.set_autopilot(True)

    while run:
        #move_forward(vehicle)
        vehicle.apply_control(actor_agent.run_step())
        time.sleep(0.05)
    print("AIUTOOOO")
    
    stop_vehicle(vehicle)
#     time.sleep(10)

finally:
    adas.destroy()
    vehicle.destroy()
    #walker.destroy()
    

Numero di veicoli trovati: 1
Veicolo distrutto: 741
Send `ACTION` to topic `carla/fcw_state`
stop
AIUTOOOO
stop
